# PyTorch Modules

### What is this notebook about?

In this notebook, we will learning about PyTorch modules and the great functionalities they provide. Later on, we'll create a small a multilayer perceptron to perform image classification on MNIST.

___

## Google Colab only!

In [1]:
# execute only if you're using Google Colab
!wget -q https://raw.githubusercontent.com/ahug/amld-pytorch-workshop/master/binder/requirements.txt -O requirements.txt
!pip install -qr requirements.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


____

In [2]:
import torch
import torch.nn as nn

print("Torch version:", torch.__version__)

Torch version: 1.5.1


In [3]:
import matplotlib.pyplot as plt

In PyTorch, there are many predefined layer like Convolutions, RNN, Pooling, Linear, etc.

These functions are wrapped in **modules** and inherit from the **torch.nn.Module** base class.

When designing a custom model in PyTorch, you should follow this strategy and derive your class from **torch.nn.Module**.

## Modules

In [4]:
print(torch.nn.Module.__doc__)

Base class for all neural network modules.

    Your models should also subclass this class.

    Modules can also contain other Modules, allowing to nest them in
    a tree structure. You can assign the submodules as regular attributes::

        import torch.nn as nn
        import torch.nn.functional as F

        class Model(nn.Module):
            def __init__(self):
                super(Model, self).__init__()
                self.conv1 = nn.Conv2d(1, 20, 5)
                self.conv2 = nn.Conv2d(20, 20, 5)

            def forward(self, x):
                x = F.relu(self.conv1(x))
                return F.relu(self.conv2(x))

    Submodules assigned in this way will be registered, and will have their
    parameters converted too when you call :meth:`to`, etc.
    


### Modules are doing a lot of "magic" under the hood.

- It registers all the parameters of your model.
- It simplifies the saving/loading of your model.
- It provides helper functions to reset/freeze/update the gradients.
- It provides helper functions to put all parameters on a device (GPU).

### What is a torch.nn.Parameter?

A Parameter is a Tensor with `requires_grad` to `True` by default, and which is automatically added to the list of parameters when used within a model.

Let's have a look at the documentation ([torch.nn.Paramter](https://pytorch.org/docs/stable/_modules/torch/nn/parameter.html))

In [5]:
print(torch.nn.Parameter.__doc__)

A kind of Tensor that is to be considered a module parameter.

    Parameters are :class:`~torch.Tensor` subclasses, that have a
    very special property when used with :class:`Module` s - when they're
    assigned as Module attributes they are automatically added to the list of
    its parameters, and will appear e.g. in :meth:`~Module.parameters` iterator.
    Assigning a Tensor doesn't have such effect. This is because one might
    want to cache some temporary state, like last hidden state of the RNN, in
    the model. If there was no such class as :class:`Parameter`, these
    temporaries would get registered too.

    Arguments:
        data (Tensor): parameter tensor.
        requires_grad (bool, optional): if the parameter requires gradient. See
            :ref:`excluding-subgraphs` for more details. Default: `True`
    


In [6]:
mod = nn.Conv1d(10, 2, 3)
print(mod.weight)

Parameter containing:
tensor([[[ 0.1103, -0.1596, -0.1015],
         [-0.0955,  0.1123, -0.0582],
         [-0.1182,  0.0353,  0.0349],
         [-0.0114, -0.0519,  0.1407],
         [ 0.1191, -0.1665,  0.0562],
         [-0.0463,  0.0737, -0.0403],
         [ 0.0940,  0.1121,  0.0919],
         [ 0.0727,  0.0484,  0.0477],
         [-0.1394,  0.1319, -0.0180],
         [ 0.0369,  0.1783, -0.0890]],

        [[ 0.0941,  0.1787, -0.1786],
         [ 0.1730, -0.0338,  0.1099],
         [-0.0972,  0.1375, -0.1051],
         [-0.1621, -0.1433, -0.1345],
         [ 0.1595, -0.0838, -0.0313],
         [-0.1148,  0.1254,  0.1246],
         [-0.0575,  0.1522, -0.1141],
         [ 0.1189,  0.1175,  0.0168],
         [ 0.0614,  0.0042, -0.0487],
         [-0.0260, -0.0392,  0.0905]]], requires_grad=True)


___

## Very simple example of a module

A module has to implement two functions:

- the `__init__` function, where you define all the layers that have learnable parameters. In the `__init__` function, you are just specifying each layer and not how it is connected to others, so it does not need to be in order of execution. Since your model's submodules and parameters are instantiated in the `__init__` function, PyTorch knows that they exist and registers them.  
Also, don't forget to always call the `super()` method.  


- the `forward` function, which is the method that defines what has to be executed during the forward pass and especially how the layers are connected. This is where you call the layers that you defined inside the `__init__` function.


In [7]:
# A simple module
class MySuperSimpleModule(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MySuperSimpleModule, self).__init__()  # Mandatory call to super
        self.linear = nn.Linear(input_size, num_classes)  # Define one Linear layer
    
    def forward(self, x):
        out = self.linear(x)
        return out

You can use the print function to list a model's submodules and parameters defined inside `init`:

In [8]:
model = MySuperSimpleModule(input_size=20, num_classes=5)
print(model)

MySuperSimpleModule(
  (linear): Linear(in_features=20, out_features=5, bias=True)
)


You can use **`model.parameters()`** to get the list of parameters of your model automatically inferred by PyTorch.

In [9]:
for name, p in model.named_parameters():  # Here we use a sligtly different version of the parameters() function
    print(name, ":\n", p)                 # which also returns the parameter name

linear.weight :
 Parameter containing:
tensor([[-0.0896,  0.2153,  0.0700, -0.0119, -0.0438,  0.0140,  0.1668, -0.0652,
         -0.1482, -0.2235,  0.0974,  0.1524,  0.2233,  0.0591, -0.0934,  0.1632,
         -0.1902, -0.1373,  0.0371, -0.0199],
        [-0.2025,  0.1954, -0.1059,  0.2122,  0.1804,  0.1338,  0.0847,  0.0297,
          0.0786, -0.1775,  0.1276, -0.0137,  0.1283,  0.0400, -0.1175, -0.0048,
          0.1317, -0.0340,  0.0383,  0.2141],
        [-0.1703, -0.0031, -0.1562, -0.1007, -0.1592, -0.0341,  0.1713, -0.0420,
         -0.0757, -0.1681, -0.0446, -0.0009, -0.0136, -0.0735, -0.1329,  0.0546,
         -0.0499,  0.0811,  0.2119,  0.1607],
        [-0.0268,  0.1423, -0.0188, -0.0128,  0.1567,  0.0234, -0.0020,  0.1162,
          0.1943, -0.2042, -0.1274,  0.1762, -0.1992, -0.1799,  0.2153,  0.0147,
         -0.1370, -0.2234, -0.0665, -0.1017],
        [-0.0425,  0.1930,  0.0375,  0.1401,  0.1810, -0.2007, -0.0250, -0.1288,
          0.1601,  0.1801, -0.2003,  0.0459,  0.

___

This intro to modules used [this medium post](https://medium.com/deeplearningbrasilia/deep-learning-introduction-to-pytorch-5bd39421c84) as a resource.